In [1]:
from T13BFLSimpCore import *

/home/cl/Documents/n-bafc/blockchain-afc/taxi


In [2]:
dat = 'trips_simpler.csv'
company = [5,4,3]#[2,3,4,5]
location = [4]#[mk for mk in range(9)]

blockchain = Blockchain()

list index out of range|Brand new blockchain, creating genesis block
BLK: C:0x7fb13#  0 <-- PH=0    :PF=0    , RL={}   , #TRXS=0
Blockchain created with ID: 0x7fb138037970 140399125690736


In [3]:
print("1 - DISTRIBUTION =================================")
import pickle
try:
    locdump = "./models/clis" + str(company) + str(location) + ".p"
    clients = pickle.load(open(locdump, "rb"))
    print("Auto-load from cache")
except:
    print("Generating dist")
    db = Distributor(dat, company, location, settings)
    db.divBoth(settings)
    clients = db.extractClients()
    import pickle
    pickle.dump(clients, open(locdump, "wb"))

1 - DISTRIBUTION =================================
Generating dist
Sum of all datalist: 1451888 | Sum of all tested: 309545
Sum of all datalist: 643211 | Sum of all tested: 132788
Sum of all datalist: 4527911 | Sum of all tested: 983132


In [4]:
print("2 - AUTHORIZATION =================================")
print("Using blockchain #" + blockchain.bc_id)
for c in clients:
    c.model = CNNLSTM(settings['frame_in'],settings['frame_out'], epochs=c.epoch)
    blockchain.authorizeClient(c)
    c.assignBlockchain(blockchain)
    print(c)

2 - AUTHORIZATION =================================
Using blockchain #0x7fb138037970
MIN:5/4--8718,   43917
CLI:4/4-64336,   23207
MIN:3/4-54806,  131480


In [5]:
print("3 - TRAINING =================================")
# Training period

for c in clients:
    c.model.setData(c.inp_train,c.out_train)
    c.train()
    c.submitModelAsTransaction()

3 - TRAINING =================================
#5/4:1.65s; MIN:5/4 MOD-->CHIDX#  1
#4/4:1.80s; CLI:4/4 MOD-->CHIDX#  1
#3/4:2.11s; MIN:3/4 MOD-->CHIDX#  1


In [6]:
print("4 - MINING =================================")
for c in clients:
    if type(c) == Miner:
        c.mine()

print("5 - CONSENSUS + INCENTIVIZATION ============")
inc = blockchain.consensus()

for c in clients:
    print(c.id, c.balance)

4 - MINING =================================
Start mining: 5/4
-8718 MINE APPROVED: POA
PREV= CURR=-8718:8.297; 64336:5.68; 54806:8.229; 
-8718 MINE APPROVED AND ACCEPTED: POQ
BLK: C:0x7fb13#  1 <-- PH=0    :PF=-8718, RL=QmR1J, #TRXS=3
Start mining: 3/4
54806 MINE APPROVED: POA
PREV= CURR=-8718:9.434; 64336:6.279; 54806:9.506; 
54806 MINE APPROVED AND ACCEPTED: POQ
BLK: C:0x7fb13#  1 <-- PH=0    :PF=54806, RL=QmR6d, #TRXS=3
5 - CONSENSUS + INCENTIVIZATION ============
{}
{'-871807179978295729': 0, '5480651930250876810': 25.218097444513482}
BLK: C:0x7fb13#  1 <-- PH=0    :PF=54806, RL=QmP6o, #TRXS=3 CONSENSUS
-871807179978295729 0
6433604349785740464 0
5480651930250876810 25.218097444513482
